<center><img src="https://imgtr.ee/images/2024/07/03/9eea693c3d3aee90f0ea2041012aa6a1.png" alt="9eea693c3d3aee90f0ea2041012aa6a1.png" border="0" /></center>

# **HiBlu**: Web Scraping

By [Mavericks Team - Hacktiv8](https://github.com/FTDS-assignment-bay/p2-final-project-mavericks) | Data Resource: [FAQ Blu](https://blubybcadigital.id/info/faq)

---

# Introduction

Program ini bertujuan untuk melakukan web scraping pada halaman FAQ website Blu BCA dan mengekstrasi informasi yang relevan untuk membuat chatbot yang mampu menjawab pertanyaan-pertanyaan umum (FAQ).

# Import Libraries

In [1]:
# Import Libraries
import pandas as pd
import time
import json

# Untuk Scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

In [20]:
# Setting untuk display kolom (opsional)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

# Web Scraping

In [2]:
# Setup webdriver (menggunakan chrome)
driver = webdriver.Chrome() 

# Membuka URL FAQ Blu
driver.get("https://blubybcadigital.id/info/faq")

# Menunggu website untuk load sampai menemukan element yang specific
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "card-faq"))) # Elemen yang specific atau yang dicari adalah "card-faq"

# Scroll page sampai bawah dengan tujuan untuk memastikan page ke-load semua
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Jeda waktu untuk page ke-load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Meng-expand jawaban yang ada di-dropdown
faq_items = driver.find_elements(By.CLASS_NAME, "question-box")
for item in faq_items:
    try:
        ActionChains(driver).move_to_element(item).click().perform() # Klik dropdown
        time.sleep(1)  # Jeda waktu agar answer ter-load
    except:
        continue

# Mendapatkan page source setelah semua jawaban ter-display
page_source = driver.page_source

# Tutup webdriver
driver.quit()

# Parse konten HTML dengan BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Ekstraksi FAQ (Pertanyaan dan jawaban)
faqs = []
faq_sections = soup.find_all('div', class_='card-faq')

for faq_section in faq_sections:
    questions = faq_section.find_all('div', class_='question-box')
    for question_box in questions:
        question_element = question_box.find('div', class_='question')
        question = question_element.get_text(strip=True) if question_element else 'Tidak ada pertanyaan'
        answer_box = question_box.find_next_sibling('div', class_='answer-box')
        if answer_box:
            answer_element = answer_box.find('div', class_='answer')
            answer = answer_element.get_text(strip=True) if answer_element else 'Tidak ada jawaban'
            faqs.append({'Question': question, 'Answer': answer})
            
# Konversi data yang diekstrak menjadi DataFrame
df = pd.DataFrame(faqs)

# Display DataFrame
print(df)


                                              Question  \
0                                         Apa itu blu?   
1                Apa perbedaan blu dengan BCA Digital?   
2                Apa perbedaan BCA Digital dengan BCA?   
3                   Apa keuntungan pakai aplikasi blu?   
4              Apakah blu punya kantor cabang offline?   
..                                                 ...   
495  Saya gagal input password dan PIN transaksi ak...   
496  Saya lupa password akun blu, apa yang harus sa...   
497  Di mana saya dapat melihat riwayat transaksi B...   
498  Di mana saya dapat melihat riwayat transaksi B...   
499  Saya ingin memutus koneksi blu dengan aplikasi...   

                                                Answer  
0    blu merupakan aplikasimobile bankingdari BCA D...  
1    blu adalah aplikasimobile bankingmilik BCA Dig...  
2    BCA Digital merupakan anak perusahaan BCA, bag...  
3    Gak terbatas ruang dan waktu, aplikasi blu bis...  
4    blu gak punya